In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
attrs = ["amount", "freq", "avg_interval"]

In [3]:
raw_data = pd.read_csv("2022AIGO_H_LAB測試資料_雲端行動科技.csv")
raw_data.dropna(axis=0, subset=["item_tag"], inplace=True)

In [43]:
cat = raw_data["item_tag"].unique()
#pd.DataFrame(cat).to_csv(r'類別.csv',encoding='utf-8-sig')

In [36]:
np.size(cat)

844

In [5]:
data_2020 = raw_data[raw_data["datetime"].str.startswith("2020")]
data_2020.loc[:, "datetime"] = pd.to_datetime(data_2020["datetime"], format="%Y-%m-%d")
data_2020["month"] = data_2020["datetime"].dt.month

C:\Users\Sylvia\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-5-bb98c96d1407>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020["month"] = data_2020["datetime"].dt.month


In [6]:
all_user = (data_2020.user_id.unique())

In [7]:
# df_2020 = data_2020[data_2020["item_tag"] != "優惠活動/折扣/集點"]
df_2020 = data_2020[data_2020["item_tag"] != "無法分類"]
df_2020 = df_2020[df_2020["item_tag"] != "餐飲需求"]

In [13]:
df_agg = df_2020.groupby(["month", "user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [14]:
df_group_alltime = df_2020.groupby(["user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [15]:

midx = pd.MultiIndex.from_product([range(1, 13), all_user])
df_output = pd.DataFrame(index = midx, columns=[f'{i}_{j}'  for i in df_2020.item_tag.unique() for j in attrs])
def make_output_matrix(x):
    (m, id) = (x.index.get_level_values("month")[0], x.index.get_level_values("user_id")[0])
    x = x.droplevel((0, 1))
    df_temp = x.stack()
    df_temp.index = [f"{x}_{y}" for x,y in df_temp.index]
    df_temp = df_temp.to_frame("").T
    df_output.loc[(m, id), df_temp.columns] = df_temp.values

df_agg.groupby(level = (0, 1)).apply(make_output_matrix)

""


In [16]:
df_output.loc[:, df_output.columns.str.endswith("avg_interval")] = df_output.loc[:, df_output.columns.str.endswith("avg_interval")].replace([0, np.nan],1000)

In [17]:
df_output.fillna(0, inplace=True)
df_output = df_output.abs()

In [18]:
df_output.to_csv("input_matrix_2020.csv")

# No group by month data

In [19]:
df_group_alltime = df_2020.groupby(["user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [25]:
df_ouput_alltime = pd.DataFrame(index = all_user, columns=[f'{i}_{j}'  for i in df_2020.item_tag.unique() for j in attrs])
def make_output_matrix(x):
    id = x.index.get_level_values("user_id")[0]
    x = x.droplevel((0))
    df_temp = x.stack()
    df_temp.index = [f"{x}_{y}" for x,y in df_temp.index]
    df_temp = df_temp.to_frame("").T
    df_ouput_alltime.loc[id, df_temp.columns] = df_temp.values

df_group_alltime.groupby(level = (0)).apply(make_output_matrix)

""


In [26]:
df_ouput_alltime.fillna(0, inplace= True)
df_ouput_alltime = df_ouput_alltime.abs()

In [27]:
from sklearn.preprocessing import MinMaxScaler
minmax_sc = MinMaxScaler()
df_ouput_alltime_scaled = minmax_sc.fit_transform(df_ouput_alltime)

In [29]:
df_ouput_alltime_scaled.to_csv("input_matrix_2020_all_scaled.csv")

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [30]:
np.savetxt("input_matrix_2020_all_scaled.csv", df_ouput_alltime_scaled, delimiter=",")